<a href="https://colab.research.google.com/github/Shayankr/AI_and_ComputerVision/blob/master/CNN_Model_Dogs_vs_Cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaggle Dataset link: https://www.kaggle.com/competitions/dogs-vs-cats/data

In [1]:
import tensorflow
from tensorflow import keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import Some Necessary Packages/Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import os
import shutil

In [4]:
# Now kaggle

In [5]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [6]:
!ls /root/.kaggle

kaggle.json


In [7]:
!ls ~/.kaggle

kaggle.json


In [8]:
import kaggle

In [9]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
!kaggle competitions download -c dogs-vs-cats

 99% 801M/812M [00:17<00:00, 71.3MB/s]
100% 812M/812M [00:17<00:00, 47.9MB/s]


In [11]:
!unzip -u dogs-vs-cats.zip

Archive:  dogs-vs-cats.zip
  inflating: sampleSubmission.csv    
  inflating: test1.zip               
  inflating: train.zip               


In [12]:
# Now, Unzip train data only
!unzip -u -q train.zip

In [13]:
len(os.listdir("train"))

25000

In [14]:
!rm -rf train_1
!rm -rf train_1/cat
!rm -rf train_1/dog

In [15]:
# Make one sepearte folder for train data and put then cat and dog images as a sepearte folder.
os.mkdir("train1")
os.mkdir("train1/cat")
os.mkdir("train1/dog")

In [16]:
src_folder = "train/"
cat_folder = "train1/cat/"
dog_folder = "train1/dog/"

In [17]:
# Now Fill cat and dog folder with cat and dog images respectively.
for image_name in os.listdir(src_folder):
  if "cat" in image_name:
    shutil.copy(src_folder+image_name, cat_folder)
  elif "dog" in image_name:
    shutil.copy(src_folder+image_name, dog_folder)
    

In [18]:
# Image Data Generator
from keras.preprocessing.image import ImageDataGenerator

In [19]:
from keras.preprocessing.image import load_img, array_to_img, img_to_array

In [20]:
idg = ImageDataGenerator(rescale=1/255.0, vertical_flip=True, horizontal_flip=True,
                         zoom_range=0.2, rotation_range=45, validation_split=0.2)

In [21]:
train_generator = idg.flow_from_directory(subset="training", directory="train1", batch_size=64, target_size=(150,150))

Found 20000 images belonging to 2 classes.


In [22]:
val_generator = idg.flow_from_directory(subset="validation", directory="train1", batch_size=64, target_size=(150,150))

Found 5000 images belonging to 2 classes.


In [23]:
!nvidia-smi  # Check whether GPU is connected or not ,,,,, if not then change runtype to GPU or TPU for faster processing.

Thu Oct  6 20:58:20 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [24]:
# Now, Buuild NN/CNN Model

In [25]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Conv2D, MaxPooling2D
from keras.activations import relu, softmax
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD

In [26]:
from keras import activations
model = Sequential()

# 1st Chunk
model.add(Conv2D(filters=32, kernel_size=(3,3), strides=(1,1), padding='valid', activation=relu,  
                 input_shape=(150,150,3)))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid', ))

# 2nd Chunk
model.add(Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='valid', activation=relu,))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid', ))

#150-3+1/1=148,148, 32

#74,74,32

#74-3+1/1 = 72,72 , 64

#36,36,64

#Apply Flattening
model.add(Flatten())
#36*36*64 = 1296*64 = 82944

#model.add(Dense(1024, activation=softmax))
#
#model.add(Dense(128, activation=softmax))
model.add(Dense(512, activation=softmax))
#
model.add(Dense(2, activation=softmax)) 
model.summary()
#RGB
# IN DNN we  have an option to intialize weights through algorithms do we have  same kind of  filtes in CNN

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 512)               4

In [27]:
# #(3,3) =9 random values 
# #32 filters
# #32*9 = 288 *3 RGB
# #288*3=864
# #864+32(bias)=896

# #we dont have any parmaters in max pooling , we dont

# #64(filters)*9(random value)=576
# #576*32(depth similar to 3 channel in  1st chunk)=18432

# #18432+ 64 (bias) = 18496

# #32
# #Maxpooling to flatten
# 36*36*64=82944
# #flatten to  Dense
# #82944 *2 +2 (bias) = 165890  #if directly last layer applied.

#we dont know the number of images in batch ,None is like default place  holder




In [28]:
# optimizers
model.compile(optimizer=SGD(), loss=categorical_crossentropy, metrics=['acc'])

In [29]:
model.fit(train_generator, epochs=20, validation_data=val_generator,) #GPU

Epoch 1/20
313/313 [==============================] - 198s 603ms/step - loss: 0.6932 - acc: 0.4987 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/20
313/313 [==============================] - 199s 636ms/step - loss: 0.6932 - acc: 0.4967 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 3/20
313/313 [==============================] - 177s 567ms/step - loss: 0.6932 - acc: 0.5003 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 4/20
313/313 [==============================] - 172s 549ms/step - loss: 0.6932 - acc: 0.4974 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 5/20
313/313 [==============================] - 187s 597ms/step - loss: 0.6932 - acc: 0.5002 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/20
313/313 [==============================] - 171s 546ms/step - loss: 0.6932 - acc: 0.4955 - val_loss: 0.6931 - val_acc: 0.5254
Epoch 7/20
313/313 [==============================] - 171s 547ms/step - loss: 0.6932 - acc: 0.5026 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 8/20
313/313 [============================

In [30]:
#task

# try to achieve accuracy of 70%

In [31]:
model.save("CatVsDogCNN_07102022")

In [32]:
# We can save tis model into Drive also. So that We can save permanently this model.
# When need to load... we can import from drive folder/file.

In [33]:
train_generator.class_indices

{'cat': 0, 'dog': 1}

In [34]:
val_generator.class_indices

{'cat': 0, 'dog': 1}

In [35]:
from tensorflow.keras.models import load_model

In [37]:
model2=load_model(filepath="CatVsDogCNN_07102022")

In [38]:
model2.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 82944)             0         
                                                                 
 dense (Dense)               (None, 512)               4

In [40]:
!zip -r  model.zip  CatVsDogCNN_07102022

  adding: CatVsDogCNN_07102022/ (stored 0%)
  adding: CatVsDogCNN_07102022/assets/ (stored 0%)
  adding: CatVsDogCNN_07102022/saved_model.pb (deflated 88%)
  adding: CatVsDogCNN_07102022/variables/ (stored 0%)
  adding: CatVsDogCNN_07102022/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: CatVsDogCNN_07102022/variables/variables.index (deflated 55%)
  adding: CatVsDogCNN_07102022/keras_metadata.pb (deflated 90%)


In [52]:
!cp model.zip /content/drive/MyDrive/"Colab Notebooks"
#shutil.copy("model.zip","/content/drive/MyDrive/'Colab Notebooks'")

"/content/drive/MyDrive/'Colab Notebooks'"

In [55]:
model.history

In [56]:
help(model.history)

Help on History in module keras.callbacks object:

class History(Callback)
 |  Callback that records events into a `History` object.
 |  
 |  This callback is automatically applied to
 |  every Keras model. The `History` object
 |  gets returned by the `fit` method of models.
 |  
 |  Example:
 |  
 |  >>> model = tf.keras.models.Sequential([tf.keras.layers.Dense(10)])
 |  >>> model.compile(tf.keras.optimizers.SGD(), loss='mse')
 |  >>> history = model.fit(np.arange(100).reshape(5, 20), np.zeros(5),
 |  ...                     epochs=10)
 |  >>> print(history.params)
 |  {'verbose': 1, 'epochs': 10, 'steps': 1}
 |  >>> # check the keys of history object
 |  >>> print(history.history.keys())
 |  dict_keys(['loss'])
 |  
 |  Method resolution order:
 |      History
 |      Callback
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  on_epoch_end(self, epoch, logs=None)
 |      Calle

In [57]:
#END